In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.auto import tqdm
from glob import glob
import time, gc
import cv2
import zipfile
import tensorflow as tf
from keras.utils.np_utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow import keras
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.models import clone_model
from keras.layers import *
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import PIL.Image as Image, PIL.ImageDraw as ImageDraw, PIL.ImageFont as ImageFont
from matplotlib import pyplot as plt
import seaborn as sns
from keras.optimizers import RMSprop
from keras.applications import VGG16
from keras.applications import VGG19

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.


Using TensorFlow backend.


In [0]:
# !pip uninstall tensorflow
# !pip install tensorflow
# !pip uninstall keras
# !pip install keras

In [0]:
inputs = Input(shape=(64,64,1))
model = Conv2D(32, (3,3), padding='same')(inputs)
model = BatchNormalization(momentum=0.5, epsilon=1e-5, gamma_initializer="uniform")(model)
model = LeakyReLU(alpha=0.1)(model)
model = Conv2D(32,  (3,3), padding='same')(model)
model = BatchNormalization(momentum=0.1, epsilon=1e-5, gamma_initializer="uniform")(model)
model = MaxPooling2D(2, 2)(model)
model = Dropout(0.2)(model)
model = Conv2D(64, (3,3), padding='same')(model)
model = BatchNormalization(momentum=0.2, epsilon=1e-5, gamma_initializer="uniform")(model)
model = LeakyReLU(alpha=0.1)(model)
model = Conv2D(64, (3,3), padding='same')(model)
model = BatchNormalization(momentum=0.1, epsilon=1e-5, gamma_initializer="uniform")(model)
model = LeakyReLU(alpha=0.1)(model)
model = MaxPooling2D(2,2)(model)
model = Dropout(0.2)(model)
model = Conv2D(128, (3,3), padding='same')(model)
model = BatchNormalization(momentum=0.2, epsilon=1e-5, gamma_initializer="uniform")(model)
model = LeakyReLU(alpha=0.1)(model)
model = Conv2D(128, (3,3), padding='same')(model)
model = BatchNormalization(momentum=0.1, epsilon=1e-5, gamma_initializer="uniform")(model)
model = LeakyReLU(alpha=0.1)(model)
model = MaxPooling2D(2,2)(model)
model = Dropout(0.2)(model)
model = Flatten()(model)
model = Dense(512)(model)
model = LeakyReLU(alpha=0.1)(model)
model = BatchNormalization()(model)
head = Dense(168, activation='softmax',name = 'head')(model)
vowel = Dense(11, activation='softmax',name = 'vowel')(model)
consonant = Dense(7, activation='softmax',name = 'consonant')(model)

model = Model(inputs=inputs, outputs=[head,vowel,consonant])

In [0]:
# from keras.utils import plot_model
# plot_model(model, to_file='model.png')
# model.summary()

In [0]:
from keras.callbacks import EarlyStopping
initial_learningrate=2e-3
batch_size = 256
epochs = 30
es = EarlyStopping(monitor='val_loss', verbose=1, patience=10)
model.compile(loss="categorical_crossentropy",
              optimizer=RMSprop(lr=initial_learningrate),
              metrics=['accuracy'])

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=10,
                                            verbose=1,
                                            factor=0.75)


In [0]:
def train_img(namelist):
    x = []
    with zipfile.ZipFile('/content/drive/My Drive/Colab Notebooks/Datasets/Bangla AI/train2.zip', 'r') as zfile:
        for i in namelist:
            data = cv2.imdecode(np.frombuffer(zfile.read(i), np.uint8), 0).astype('float32')
            data = data/255
            x.append(data)
    
    return x

def test_img():
    x = []
    with zipfile.ZipFile('/content/drive/My Drive/Colab Notebooks/Datasets/Bangla AI/test.zip', 'r') as zfile:
        name = zfile.namelist()
        for i in name:
            data = cv2.imdecode(np.frombuffer(zfile.read(i), np.uint8), 0).astype('float32')
            data = data/255
            x.append(data)
    
    return x

In [0]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Datasets/Bangla AI/train.csv")
df.head()
ls = df['image_id'].tolist()
ls = [i+'.png' for i in ls ]
# x = ls[:40000]
# xx = ls[len(x):80000]
# xxx = ls[80000:120000]
# xxxx= ls[120000:160000]
# xxxxx = ls[160000:]
z = df.iloc[:,1:4].values.tolist()
# y = z[:40000]
# yy = z[40000:80000]
# yyy = z[80000:120000]
# yyyy = z[120000:160000]
# yyyyy = z[160000:]

label = z
name = ls

In [0]:
class MultiOutputDataGenerator(keras.preprocessing.image.ImageDataGenerator):

    def flow(self,
             x,
             y=None,
             batch_size=32,
             shuffle=True,
             sample_weight=None,
             seed=None,
             save_to_dir=None,
             save_prefix='',
             save_format='png',
             subset=None):

        targets = None
        target_lengths = {}
        ordered_outputs = []
        for output, target in y.items():
            if targets is None:
                targets = target
            else:
                targets = np.concatenate((targets, target), axis=1)
            target_lengths[output] = target.shape[1]
            ordered_outputs.append(output)


        for flowx, flowy in super().flow(x, targets, batch_size=batch_size,
                                         shuffle=shuffle):
            target_dict = {}
            i = 0
            for output in ordered_outputs:
                target_length = target_lengths[output]
                target_dict[output] = flowy[:, i: i + target_length]
                i += target_length

            yield flowx, target_dict

In [0]:
datagen = MultiOutputDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=8,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range = 0.15, # Randomly zoom image 
    width_shift_range=0.15,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.15,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False)  # randomly flip images

In [0]:
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import load_model

model = load_model('/content/drive/My Drive/Colab Notebooks/kan4model.h5')
train_image = np.array(train_img(name))
train_image = train_image.reshape(train_image.shape[0], train_image.shape[1], train_image.shape[2], 1)
label = np.array(label)
head = np.array(label[:,0].tolist())
vowel = np.array(label[:,1].tolist())
consonant = np.array(label[:,2].tolist())
head = to_categorical(head, num_classes = 168)
vowel = to_categorical(vowel,num_classes=11)
consonant = to_categorical(consonant,num_classes=7)

x_train, x_test, y_train_root, y_test_root, y_train_vowel, y_test_vowel, y_train_consonant, y_test_consonant = train_test_split(train_image, head, vowel, 
                                                                                                                                consonant, test_size=0.08, random_state=666)
datagen.fit(x_train)


In [0]:

history = model.fit_generator(datagen.flow(x_train, {'head': y_train_root, 'vowel': y_train_vowel, 'consonant': y_train_consonant}, batch_size=batch_size),
                          epochs = epochs, validation_data = (x_test, [y_test_root, y_test_vowel, y_test_consonant]), 
                          steps_per_epoch=x_train.shape[0] // batch_size, 
                          callbacks=[learning_rate_reduction,es])
model.save('/content/drive/My Drive/Colab Notebooks/kan4model.h5')


Epoch 1/30
721/721 [==============================] - 237s 329ms/step - loss: 0.3657 - head_loss: 0.2014 - vowel_loss: 0.0854 - consonant_loss: 0.0788 - head_accuracy: 0.9393 - vowel_accuracy: 0.9744 - consonant_accuracy: 0.9752 - val_loss: 0.3844 - val_head_loss: 0.2331 - val_vowel_loss: 0.0790 - val_consonant_loss: 0.0727 - val_head_accuracy: 0.9416 - val_vowel_accuracy: 0.9816 - val_consonant_accuracy: 0.9806
Epoch 2/30
721/721 [==============================] - 237s 329ms/step - loss: 0.3618 - head_loss: 0.1991 - vowel_loss: 0.0840 - consonant_loss: 0.0787 - head_accuracy: 0.9394 - vowel_accuracy: 0.9749 - consonant_accuracy: 0.9756 - val_loss: 0.3891 - val_head_loss: 0.2292 - val_vowel_loss: 0.0851 - val_consonant_loss: 0.0750 - val_head_accuracy: 0.9430 - val_vowel_accuracy: 0.9790 - val_consonant_accuracy: 0.9800
Epoch 3/30
721/721 [==============================] - 237s 329ms/step - loss: 0.3587 - head_loss: 0.1978 - vowel_loss: 0.0830 - consonant_loss: 0.0779 - head_accuracy: 

In [0]:
del train_image
del vowel
del consonant
del head
del x_train

In [0]:
x = np.array(test_img())

x = x.reshape(x.shape[0],64,64,1)

z = model.predict(x)

x = np.argmax(z[0], axis=1)
y = np.argmax(z[1], axis=1)
z = np.argmax(z[2], axis=1)
print(x)